<a href="https://colab.research.google.com/github/samsambutdifferent/Charactorise/blob/master/frame_based_data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frame Based Data Loader

## Full Data Load

In [1]:
# %pip install tensorflow imutils numpy imageio opencv-python tqdm

# from imutils import paths
from tqdm import tqdm
import pandas as pd 
# import numpy as np
import shutil
# import cv2
import os

### Data collection

Will download
- UCF101.rar
- UCF101TrainTestSplits-RecognitionTask.zip

before unzipping them to:
- ucfTrainTestlist/
- data/

only needs to be run once

In [2]:
# !wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
# !wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

unzip the files:

for macs:
*!brew install rar*

then allow the app to run

-> apple button

-> system settings

-> privacy & secruity

-> scroll to bottom and click

In [3]:
%%capture

## for macs:
# !brew install rar

!unrar e UCF101.rar data/
!unzip -qq UCF101TrainTestSplits-RecognitionTask.zip

## Metadata Loading

In [3]:
# Open the .txt file which have names of training videos
f = open("ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# Create a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [4]:
# Open the .txt file which have names of test videos
with open("ucfTrainTestlist/testlist01.txt", "r") as f:
    temp = f.read()
videos = temp.split("\n")

# Create a dataframe having video names
test = pd.DataFrame()
test["video_name"] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


## Utility Functions

In [14]:
def extract_tag(video_path):
    return video_path.split("/")[0]

def separate_video_name(video_name):
    return video_name.split("/")[1]

def rectify_video_name(video_name):
    return video_name.split(" ")[0]

def move_videos(df, output_dir):
    if not os.path.exists("data"):
        os.mkdir("data")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

## DataFrame Preparation

In [6]:
train["tag"] = train["video_name"].apply(extract_tag)
train["video_name"] = train["video_name"].apply(separate_video_name)
train.head()

,video_name,tag
0,v_ApplyEyeMakeup_g08_c01.avi 1,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g08_c02.avi 1,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g08_c03.avi 1,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g08_c04.avi 1,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g08_c05.avi 1,ApplyEyeMakeup


In [7]:
train["video_name"] = train["video_name"].apply(rectify_video_name)
train.head()

,video_name,tag
0,v_ApplyEyeMakeup_g08_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g08_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g08_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g08_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g08_c05.avi,ApplyEyeMakeup


In [8]:
test["tag"] = test["video_name"].apply(extract_tag)
test["video_name"] = test["video_name"].apply(separate_video_name)
test.head()

,video_name,tag
0,v_ApplyEyeMakeup_g01_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g01_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g01_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g01_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g01_c05.avi,ApplyEyeMakeup


## Filtering Top-n Actions

In [9]:
n = 10
topNActs = train["tag"].value_counts().nlargest(n).reset_index()["index"].tolist()
train_new = train[train["tag"].isin(topNActs)]
test_new = test[test["tag"].isin(topNActs)]
train_new.shape, test_new.shape

((1171, 2), (459, 2))

In [10]:
train_new.reset_index(inplace = True,drop = True)
test_new.reset_index(inplace = True,drop = True)

In [16]:
train_new

,video_name,tag
0,v_BoxingPunchingBag_g08_c01.avi,BoxingPunchingBag
1,v_BoxingPunchingBag_g08_c02.avi,BoxingPunchingBag
2,v_BoxingPunchingBag_g08_c03.avi,BoxingPunchingBag
3,v_BoxingPunchingBag_g08_c04.avi,BoxingPunchingBag
4,v_BoxingPunchingBag_g08_c05.avi,BoxingPunchingBag
...,...,...
1166,v_TennisSwing_g25_c02.avi,TennisSwing
1167,v_TennisSwing_g25_c03.avi,TennisSwing
1168,v_TennisSwing_g25_c04.avi,TennisSwing
1169,v_TennisSwing_g25_c05.avi,TennisSwing


In [15]:
def move_videos(df, output_dir):
    if not os.path.exists("data"):
        os.mkdir("data")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

move_videos(train_new, "train")
move_videos(test_new, "test")

  0%|          | 0/1171 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/v_BoxingPunchingBag_g08_c01.avi'

In [ ]:
train_new.to_csv("train.csv", index=False)
test_new.to_csv("test.csv", index=False)